In [ ]:
from tensorflow.keras import layers, Model
from tensorflow.keras import regularizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, BatchNormalization
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
import keras
import sys
import pandas as pd
import sklearn as sk
import numpy as np

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU')) > 0
print(tf.config.list_physical_devices('GPU'))
print("GPU is", "available" if gpu else "NOT AVAILABLE")

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     # Asignar 8GB a cada modelo (24GB total / 3 modelos)
#     tf.config.set_logical_device_configuration(
#         gpus[0],
#         [tf.config.LogicalDeviceConfiguration(memory_limit=10000)]  # 8GB en MB
#     )
#     print("GPU limitada a 10GB para este notebook.")

In [ ]:
# SEMILLA
tf.random.set_seed(42)

In [ ]:
# Parámetros
forma_entrada = (224, 224, 3)
tamaño_lote = 64
épocas = 60
initial_learning_rate = 1e-3

preprocess_fn = tf.keras.applications.vgg16.preprocess_input


# Rutas
ruta_entrenamiento = './datasets/2D/data_augmentation/color_2/train'
ruta_validacion = './datasets/2D/data_augmentation/color_2/validation'
ruta_test = './datasets/2D/data_augmentation/color_2/test'

# ruta_entrenamiento = './datasets/order_data/color/train'
# ruta_validacion = './datasets/order_data/color/validation'
# ruta_test = './datasets/order_data/color/test'

# Generadores
datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_fn,
    rescale=1./255
)
datagen_valid = ImageDataGenerator(
    # preprocessing_function=preprocess_fn,
    rescale=1./255
) # ANTES APLIQUE RESCALE
datagen_test = ImageDataGenerator(
    # preprocessing_function=preprocess_fn,
    rescale=1./255
)

generador_entrenamiento = datagen.flow_from_directory(
    ruta_entrenamiento,
    target_size=forma_entrada[:2],
    batch_size=tamaño_lote,
    shuffle=True,
    class_mode='categorical'
)

generador_validacion = datagen_valid.flow_from_directory(
    ruta_validacion,
    target_size=forma_entrada[:2],
    batch_size=tamaño_lote,
    shuffle=False,
    class_mode='categorical'
)

generador_test = datagen_test.flow_from_directory(
    ruta_test,
    target_size=forma_entrada[:2],
    batch_size=tamaño_lote,
    shuffle=False,
    class_mode='categorical'
)


In [ ]:
X, _ = next(generador_entrenamiento)
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)  # Nota: i+1 para que los índices empiecen en 1
    plt.imshow(X[i])  # Añade cmap='gray' para imágenes en escala de grises
    plt.axis('off')
plt.suptitle('Ejemplos con Cutout aplicado aleatoriamente')
plt.show()

In [ ]:
steps = len(generador_entrenamiento.filepaths) // generador_entrenamiento.batch_size
steps

In [ ]:
##### AÑADIR LR DECAY FACTOR #####
# learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=initial_learning_rate,
#     decay_steps=30*steps,  # Decaimiento más frecuente
#     decay_rate=0.9,  # Decaimiento más fuerte
#     staircase=True)
learning_rate = initial_learning_rate

In [ ]:
# !pip install tensorflow-addons

In [ ]:
# latest = "models/VGG16/siamese_model_vgg16_apple_data_REF.h5"

# base_model = tf.keras.models.load_model(latest, compile=False)

# base_model.summary()

In [ ]:
# # Extraer solo la parte VGG16 del modelo cargado
# vgg_layer = base_model.get_layer('vgg16')  # Asumiendo que se llama 'vgg16' en tu modelo cargado
# vgg_layer.trainable = False  # Congelar pesos

# entrada = layers.Input(shape=forma_entrada)
# x = vgg_layer(entrada)  # Usar la VGG16 cargada
# x = layers.Flatten()(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
# x = layers.Dropout(0.75)(x)
# salida = layers.Dense(4, activation='softmax')(x)

# # 3. Compilar
# model = Model(inputs=entrada, outputs=salida)
# optimizador = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# model.compile(
#     loss='categorical_crossentropy', 
#     optimizer=optimizador, 
#     metrics=['accuracy']
# )
# #
# model.summary()

In [ ]:
# Construccion del modelo
# --------------------------------------------------------------------------
def model_init(learning_rate, dropout_rate, l2_reg):
    base_model = VGG16(
        weights='imagenet', 
        include_top=False, 
        input_shape=forma_entrada
    )
    
    for layer in base_model.layers:
        layer.trainable = False
        
    entrada = layers.Input(shape=forma_entrada)
    x = base_model(entrada)
    x = layers.Flatten()(x)
    # x = BatchNormalization()(x)
    x = layers.Dense(
        256,
        activation='relu',
        kernel_regularizer=regularizers.l2(l2_reg)
    )(x)
    x = layers.Dropout(dropout_rate)(x)
    salida = layers.Dense(4, activation='softmax')(x)
    
    model = Model(inputs=entrada, outputs=salida)
    optimizador = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='categorical_crossentropy', 
        optimizer=optimizador, 
        metrics=['accuracy']
    )
    return model

from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split
import numpy as np

# 1. Definir función objetivo a optimizar
def optimize_model(lr, dropout_rate, l2_reg, batch_size):
    # Convertir parámetros
    lr = 10**lr  # Usar escala logarítmica
    l2_reg = 10**l2_reg
    batch_size = int(batch_size)
    
    # Construir modelo con hiperparámetros actuales
    modelo = model_init(
        learning_rate=lr,
        dropout_rate=dropout_rate,
        l2_reg=l2_reg
    )
    
    # Entrenamiento reducido para evaluación rápida
    history = modelo.fit(
        generador_entrenamiento_combinado,
        #steps_per_epoch=train_samples // tamaño_lote,  # Subconjunto para optimización
        validation_data=generador_validacion_combinado,
        #validation_steps=val_samples // tamaño_lote,
        epochs=3,
        verbose=1
    )
    
    # Retornar el mejor valor de val_accuracy
    return np.max(history.history['val_accuracy'])

# 2. Definir espacio de búsqueda
pbounds = {
    'lr': (-5, -3),       # 10^-5 a 10^-3
    'dropout_rate': (0.3, 0.7),
    'l2_reg': (-5, -2),   # 10^-5 a 10^-2
    'batch_size': (16, 64) # Discretizar después
}

# 3. Crear optimizador bayesiano
optimizer = BayesianOptimization(
    f=optimize_model,
    pbounds=pbounds,
    random_state=42,
    verbose=1
)

# 4. Ejecutar optimización
optimizer.maximize(
    init_points=10,  # Exploración inicial aleatoria
    n_iter=2,      # Iteraciones bayesianas
)

# Obtener mejores parámetros
best_params = optimizer.max['params']
best_params['batch_size'] = int(best_params['batch_size'])  # Discretizar

# Construir modelo final con mejores parámetros
model = model_init(
    learning_rate=10**best_params['lr'],
    dropout_rate=best_params['dropout_rate'],
    l2_reg=10**best_params['l2_reg']
)

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split
import numpy as np

# 1. Definir función objetivo a optimizar
def optimize_model(lr, dropout_rate, l2_reg):
    # Convertir parámetros
    lr = 10**lr  # Usar escala logarítmica
    l2_reg = 10**l2_reg

    # Construir modelo con hiperparámetros actuales
    modelo = model_init(
        learning_rate=lr,
        dropout_rate=dropout_rate,
        l2_reg=l2_reg
    )
    
    # Entrenamiento reducido para evaluación rápida
    history = modelo.fit(
        generador_entrenamiento,
        # steps_per_epoch=train_samples // tamaño_lote,  # Subconjunto para optimización
        validation_data=generador_validacion,
        # validation_steps=val_samples // tamaño_lote,
        epochs=3,
        verbose=1
    )
    
    # Retornar el mejor valor de val_accuracy
    return np.max(history.history['val_accuracy'])

# 2. Definir espacio de búsqueda
pbounds = {
    'lr': (-5, -2),       # 10^-5 a 10^-3
    'dropout_rate': (0.7, 0.9),
    'l2_reg': (-5, -2),   # 10^-5 a 10^-2
}

# 3. Crear optimizador bayesiano
optimizer = BayesianOptimization(
    f=optimize_model,
    pbounds=pbounds,
    random_state=42,
    verbose=1
)

# 4. Ejecutar optimización
optimizer.maximize(
    init_points=2,  # Exploración inicial aleatoria
    n_iter=10,      # Iteraciones bayesianas
)

In [ ]:
optimizer.max['params']

In [ ]:
# Obtener mejores parámetros
best_params = optimizer.max['params']

# Construir modelo final con mejores parámetros
model = model_init(
    learning_rate=10**best_params['lr'],
    dropout_rate=best_params['dropout_rate'],
    l2_reg=10**best_params['l2_reg']
)

In [ ]:
#model = model_init(learning_rate, 0.7, 1e-4)

In [ ]:
# Configuración de callbacks
import os
path_models = './models/pruebas/VGG16_RGB_bayesian_batchdef/'
os.makedirs(path_models, exist_ok=True)
arch = 'VGG16_SINGLE_INPUT'

In [ ]:
# Ruta donde se guardará el archivo (ej: 'C:/Users/tu_usuario/proyecto/resultados/hiperparametros.txt')
ruta_archivo = path_models + "hiperparametros.txt"  # ¡Cambia esto!

# Asegurarse de que la carpeta exista (si no, la crea)
os.makedirs(os.path.dirname(ruta_archivo), exist_ok=True)

# Guardar en formato legible (clave = valor)
with open(ruta_archivo, "w") as f:
    for key, value in best_params.items():
        f.write(f"{key} = {value}\n")

print(f"¡Hiperparámetros guardados en: {ruta_archivo}!")

In [ ]:
checkpoint = ModelCheckpoint(
    path_models + "model_single_input_silhouette_0_0.h5",
    monitor='val_loss', # <-- Guarda el mejor modelo basado en val_loss (más estable que val_accuracy)
    verbose=1,
    save_best_only=True, # guarda cuando haya mejoras
    save_weights_only=True,
    mode='min', #estaba en auto
    period=10  # Guarda pesos cada 10 épocas
)

early = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001, 
    patience=10,  # <-- Detén el entrenamiento cuando val_accuracy no mejore en 10 épocas
    verbose=1,
    restore_best_weights=True,
    mode='max'
)
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Monitorear pérdida de validación
    mode='min',         # Reducir LR cuando val_loss deje de disminuir
    factor=0.5,         # Reducción moderada del LR
    patience=5,        # Esperar 5 épocas sin mejora
    verbose=1,
    min_lr=1e-6         # LR mínimo permitido+
)

# from tensorflow.keras.callbacks import ReduceLROnPlateau

# lr_scheduler = ReduceLROnPlateau(
#     monitor='val_accuracy',
#     factor=0.5,  # Reducir LR a la mitad
#     patience=10,  # Esperar 10 épocas sin mejora
#     min_lr=1e-6,  # LR mínimo
#     verbose=1
# )

# Añadir a los callbacks
callbacks = [checkpoint, reduce_lr, early]  # <-- Añadido

In [ ]:
# Para TensorFlow 2.6 (compatible con CUDA 11.2 y cuDNN 8.1)
# %pip uninstall tensorflow -y
# %pip install tensorflow-gpu==2.6.0

In [ ]:
historia = model.fit(
    generador_entrenamiento,
    #steps_per_epoch=generador_entrenamiento.samples // tamaño_lote,
    epochs=épocas,
    validation_data=generador_validacion,
    #validation_steps=generador_validacion.samples // tamaño_lote,
    callbacks=callbacks,
    # class_weight=class_weights  # <-- Añadir esto
)

model.save(path_models + arch + '/model_single_input_silhouette_0_0.h5')

In [ ]:
current_lr = model.optimizer.lr.numpy()
print(f"Learning rate actual: {current_lr}")

# Graficar el learning rate a lo largo de las épocas
plt.plot(historia.history['lr'])
plt.title('Learning Rate durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('LR')
plt.savefig(path_models +"VGG16_silhouette_0_0_lr_plot.png")
plt.show()

In [ ]:
model.save(path_models + arch + '/model_single_input_silhouette_0_0.h5')

In [ ]:
# Gráfico de Pérdida
plt.figure(figsize=(7,5))
plt.plot(historia.history['loss'], label='Training Loss')
plt.plot(historia.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Loss Plot")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.savefig(path_models +"VGG16_silhouette_0_0_loss_plot.png")  # Guarda el gráfico en loss_plot.png
plt.show()

# Gráfico de Precisión
plt.figure(figsize=(7,5))
plt.plot(historia.history['accuracy'], label='Training Accuracy')
plt.plot(historia.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Accuracy Plot")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.savefig(path_models +"VGG16_silhouette_0_0_accuracy_plot.png")  # Guarda el gráfico en accuracy_plot.png
plt.show()

In [ ]:
# # # #### USAR PESOS YA GUARDADOS
# from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# latest = path_models + arch + '/model_single_input_silhouette_0_0.h5'
# modelo = tf.keras.models.load_model(latest)
# modelo.summary()

# # Evaluación sobre el set de prueba
# y_pred = modelo.predict(generador_test, verbose=1)
# predicted_labels = np.argmax(y_pred, axis=1)
# true_labels = generador_test.classes


# categorias = ['Class 1', 'Class 2', 'Class 3', 'Class 4']

# #class_names = ["clase_1", "clase_2", "clase_3", "clase_4"]
# # class_names = ["clase_1", "clase_2"]
# # Calcular métricas
# print('Accuracy:', accuracy_score(true_labels, y_pred))

# from sklearn.metrics import classification_report

# # Suponiendo que tienes:
# # verdaderas: etiquetas verdaderas
# # predichas: etiquetas predichas
# # categorias: nombres de las clases

# # Generar el reporte
# reporte = classification_report(true_labels, y_pred, target_names=categorias)
# print(reporte)
# # Guardar en un archivo .txt
# with open(path_models+'reporte_clasificacion.txt', 'w') as archivo:
#     archivo.write(reporte)

In [ ]:
#####Check Current Learning Rate######

#current_lr = learning_rate(model.optimizer.iterations)
#print(f"Current learning rate: {current_lr.numpy()}")

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Evaluación sobre el set de prueba
y_pred = model.predict(generador_test, verbose=1)
predicted_labels = np.argmax(y_pred, axis=1)
true_labels = generador_test.classes

# Calcular métricas
cm = confusion_matrix(true_labels, predicted_labels)
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average=None)
recall = recall_score(true_labels, predicted_labels, average=None)
f1 = f1_score(true_labels, predicted_labels, average=None)

categories = ['Class 1', 'Class 2', 'Class 3', 'Class 4']

# Crear la gráfica de la matriz de confusión
sns.set(font_scale=1.4)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='g', cmap='Pastel1_r', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predictions')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.savefig(path_models + "VGG16_silhouette_0_0_confusion_matrix.png")  # Guarda la imagen en un archivo PNG
plt.close()  # Cierra la figura

class_counts = cm.sum(axis=1)  # número total de instancias reales por clase
diag = np.diag(cm)             # verdaderos positivos por clase
class_accuracy = diag / class_counts

# Tabla en pandas
df = pd.DataFrame({
    'Clase': categories,
    'Accuracy': class_accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1
})

# Imprimir en pantalla (opcional)
print(f"Accuracy: {accuracy}")
print(df)
print("Average Precision:", df["Precision"].mean())
print("Average Recall:", df["Recall"].mean())
print("Average F1-Score:", df["F1-Score"].mean())

# Guardar las métricas en un archivo de texto
with open(path_models + "VGG16_silhouette_0_0_metrics.txt", "w") as f:
    f.write(f"Accuracy: {accuracy}\n\n")
    f.write("Confusion Matrix:\n")
    f.write(np.array2string(cm))
    f.write("\n\nMetrics Table:\n")
    f.write(df.to_string(index=False))
    f.write("\n\n")
    f.write(f"Average Precision: {df['Precision'].mean()}\n")
    f.write(f"Average Recall: {df['Recall'].mean()}\n")
    f.write(f"Average F1-Score: {df['F1-Score'].mean()}\n")
